<a href="https://colab.research.google.com/github/jeaneigsi/cookbook/blob/main/LlamaCpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://llama-cpp-python.readthedocs.io/en/latest/api-reference/?ref=localhost#llama_cpp.Llama

# Nouvelle section

In [1]:
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu
!pip install llama-cpp-python

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.1 MB/s eta 0:00:00


In [2]:
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

In [2]:
!pip3 install hf_transfer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 32.0 MB/s eta 0:00:00


In [3]:
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False


downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf to /root/.cache/huggingface/hub/tmpj417pxf1
mistral-7b-instruct-v0.2.Q5_K_M.gguf:  62% 3.18G/5.13G [00:39<00:31, 61.9MB/s]^C


In [5]:
!pip3 install huggingface-hub

In [6]:
prompt= """
### System:
You are an AI assistant that follows instructions exceptionally well. Be as helpful as possible.
### User:
List the top 10 APIs
### Assistant:
"""

In [ ]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="/content/mistral-7b-instruct-v0.2.Q5_K_M.gguf",  # Download the model file first
  n_ctx= 10000, # The max sequence length to use - note that longer sequence lengths require much more resource
  n_batch=512,
  n_gpu_layers=-1
)

In [8]:
print(llm.model)

98917451199920


In [9]:
output = llm(
   "Q: Who is the president of USA? A: ",
   max_tokens=20000,
   stop=["Q:", "\n"],
   echo=True
)
print(output)


llama_print_timings:        load time =    7378.75 ms
llama_print_timings:      sample time =      31.72 ms /    59 runs   (    0.54 ms per token,  1859.79 tokens per second)
llama_print_timings: prompt eval time =    7378.60 ms /    13 tokens (  567.58 ms per token,     1.76 tokens per second)
llama_print_timings:        eval time =   45555.18 ms /    58 runs   (  785.43 ms per token,     1.27 tokens per second)
llama_print_timings:       total time =   53169.71 ms /    71 tokens


{'id': 'cmpl-730faab7-2954-4d49-9776-23cdc730f3fc', 'object': 'text_completion', 'created': 1713689420, 'model': '/content/mistral-7b-instruct-v0.2.Q5_K_M.gguf', 'choices': [{'text': 'Q: Who is the president of USA? A:  As of my knowledge up to 2021, the President of the United States is Joe Biden. He assumed office on January 20, 2021. However, please check the most recent and reliable sources for the current information as political situations can change rapidly.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 58, 'total_tokens': 71}}


In [10]:
output=output['choices'][0]['text']
print(output)

Q: Who is the president of USA? A:  As of my knowledge up to 2021, the President of the United States is Joe Biden. He assumed office on January 20, 2021. However, please check the most recent and reliable sources for the current information as political situations can change rapidly.


In [ ]:
!pip install langchain_community langchain_core langchain

In [12]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [13]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate.from_template(template)

In [14]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [16]:
question = """
Question: A rap battle between Stephen Colbert and John Oliver
"""
llm.invoke(question)

Answer: (Verse 1 - Stephen Colbert)
Yo, it's Colbert, the king of satire,
I've got a mouth full of wit, like a lard bucket,
But don't get it twisted, I may be a clown, but I'll knock your crown off.

(Verse 2 - John Oliver)
Stephen, it's John Oliver, the underdog with the upper hand,
You may have a mouth full of words, but I've got a brain full of facts,
So while you're busy clowning around, I'll be over here, delivering truth bombs.

(Verse 3 - Stephen Colbert)
John, you may think you've got the upper hand,
But let me remind you, it's not about who can spit the most facts,
It's about who can make people laugh and think at the same time,
And when it comes to that, I'm still the king of satire.

(Verse 4 - John Oliver)
Stephen, I may have given you the microphone for this round,
But let me make one thing clear, when it comes to rap battles, I may not be the reigning champion,
But I guarantee you, if you step into the ring with me, it's gonna be a battle royale of epic proportions.

(Out


llama_print_timings:        load time =    4972.11 ms
llama_print_timings:      sample time =     287.15 ms /   496 runs   (    0.58 ms per token,  1727.30 tokens per second)
llama_print_timings: prompt eval time =    8842.10 ms /    16 tokens (  552.63 ms per token,     1.81 tokens per second)
llama_print_timings:        eval time =  384802.85 ms /   495 runs   (  777.38 ms per token,     1.29 tokens per second)
llama_print_timings:       total time =  396726.85 ms /   511 tokens


"Answer: (Verse 1 - Stephen Colbert)\nYo, it's Colbert, the king of satire,\nI've got a mouth full of wit, like a lard bucket,\nBut don't get it twisted, I may be a clown, but I'll knock your crown off.\n\n(Verse 2 - John Oliver)\nStephen, it's John Oliver, the underdog with the upper hand,\nYou may have a mouth full of words, but I've got a brain full of facts,\nSo while you're busy clowning around, I'll be over here, delivering truth bombs.\n\n(Verse 3 - Stephen Colbert)\nJohn, you may think you've got the upper hand,\nBut let me remind you, it's not about who can spit the most facts,\nIt's about who can make people laugh and think at the same time,\nAnd when it comes to that, I'm still the king of satire.\n\n(Verse 4 - John Oliver)\nStephen, I may have given you the microphone for this round,\nBut let me make one thing clear, when it comes to rap battles, I may not be the reigning champion,\nBut I guarantee you, if you step into the ring with me, it's gonna be a battle royale of epi

In [17]:
llm_chain = prompt | llm

In [18]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
llm_chain.invoke({"question": question})

Llama.generate: prefix-match hit




First, we need to find out when Justin Bieber was born. A quick internet search will reveal that he was born on March 1, 1994.

Now, let's find out which NFL team won the Super Bowl in the year Justin Bieber was born.

The Super Bowl for the year 1994 was held on January 30, 1994. The teams that made it to the Super Bowl were the Buffalo Bills and the Dallas Cowboys.

In the end, it was the Dallas Cowboys who emerged victorious in Super Bowl XXVIII against the Buffalo Bills.

So, there you have it! The NFL team that won the Super Bowl in the year Justin Bieber was born was the Dallas Cowboys.


llama_print_timings:        load time =    4972.11 ms
llama_print_timings:      sample time =      95.70 ms /   172 runs   (    0.56 ms per token,  1797.21 tokens per second)
llama_print_timings: prompt eval time =   25850.84 ms /    45 tokens (  574.46 ms per token,     1.74 tokens per second)
llama_print_timings:        eval time =  132042.80 ms /   171 runs   (  772.18 ms per token,     1.30 tokens per second)
llama_print_timings:       total time =  158930.45 ms /   216 tokens


"\n\nFirst, we need to find out when Justin Bieber was born. A quick internet search will reveal that he was born on March 1, 1994.\n\nNow, let's find out which NFL team won the Super Bowl in the year Justin Bieber was born.\n\nThe Super Bowl for the year 1994 was held on January 30, 1994. The teams that made it to the Super Bowl were the Buffalo Bills and the Dallas Cowboys.\n\nIn the end, it was the Dallas Cowboys who emerged victorious in Super Bowl XXVIII against the Buffalo Bills.\n\nSo, there you have it! The NFL team that won the Super Bowl in the year Justin Bieber was born was the Dallas Cowboys."

In [19]:
!nproc --all #CONNAITRE LE NOMBRE DE THREADS


2


In [ ]:
# other test

# Explore with PowerInfer



```
# https://github.com/SJTU-IPADS/PowerInfer
```



In [22]:
!git clone https://github.com/SJTU-IPADS/PowerInfer
%cd PowerInfer
!pip install -r requirements.txt # install Python helpers' dependencies

Cloning into 'PowerInfer'...
remote: Enumerating objects: 8824, done.
remote: Counting objects: 100% (607/607), done.
remote: Compressing objects: 100% (360/360), done.
remote: Total 8824 (delta 328), reused 483 (delta 237), pack-reused 8217
Receiving objects: 100% (8824/8824), 12.19 MiB | 16.48 MiB/s, done.
Resolving deltas: 100% (6142/6142), done.
/content/PowerInfer/PowerInfer/PowerInfer
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get

In [ ]:
!cmake -S . -B build
!cmake --build build --config Release

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download --resume-download --local-dir ReluLLaMA-7B --local-dir-use-symlinks False PowerInfer/ReluLLaMA-7B-PowerInfer-GGUF

In [25]:
#./build/bin/main -m /PATH/TO/MODEL -n $output_token_count -t $thread_num -p $prompt

In [26]:
!./build/bin/main -m /content/PowerInfer/ReluLLaMA-7B/llama-7b-relu.powerinfer.gguf -n 128 -t 2 -p "Once upon a time"

/bin/bash: line 1: ./build/bin/main: No such file or directory


Conclusion : **Extremement lent PowerInfer**